In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from pathlib import Path
# from tqdm.notebook import tqdm
from imageio import imread

import seaborn as sns


In [5]:
def read_grayscale_pngs(path, width=20, height=13):
    path = Path(path)
    if not path.exists():
        print("Path {} doesn't exist".format(path))
        return None

    # print(len([name for name in os.listdir('{}/.'.format(path)) if os.path.isfile(name)]))
    num_files = len(list(path.glob('**/*.png'))) # Calculate amount of files in directory
    # num_files = len([f for f in path.iterdir() if path.joinpath(f).is_file()]) # Calculate amount of files in directory

    images = np.empty((num_files, 13, 20))

    for i, image_path in enumerate(sorted(path.glob('**/*.png'), key=lambda f: int(f.stem))):
        images[i] = np.array(imread(image_path))[:, :, 0] # Pixel data: It's grayscale so take only Red values from [R, G, B, A]
    return images

In [34]:
legal = read_grayscale_pngs("out/legal/orig")
illegal = read_grayscale_pngs("out/illegal/orig")  

mirrored_legal = read_grayscale_pngs("out/legal/mirrored")
mirrored_illegal = read_grayscale_pngs("out/illegal/mirrored")  

legal_test = read_grayscale_pngs("testing/legal")
illegal_test = read_grayscale_pngs("testing/illegal")


In [35]:
from sklearn.model_selection import train_test_split# Spliiting data into test and train sets


X_train = np.concatenate((legal, illegal))
X_train = X_train / 255.0
Y_train = np.concatenate((np.full(len(legal), 0), np.full(len(illegal), 1)))

# X_mirror = np.concatenate((mirrored_legal, mirrored_illegal))
# X_mirror = X_mirror / 255.0
# Y_mirror = np.concatenate((np.full(len(mirrored_legal), 0), np.full(len(mirrored_illegal), 1))


X_test = np.concatenate((legal_test, illegal_test))
X_test = X_test / 255.0
Y_test = np.concatenate((np.full(len(legal_test), 0), np.full(len(illegal_test), 1)))

# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)# fitting the model

In [24]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
# train_dataset = train_dataset.shuffle(buffer_size=len(X_train), reshuffle_each_iteration=True, seed=133742).batch(10)


In [33]:
# Convolutional
keras.backend.clear_session()
modelr = keras.Sequential()

modelr.add(layers.Reshape((1,260), input_shape=(13,20)))
# modelr.add(layers.Flatten())
modelr.add(layers.LSTM(20))
modelr.add(layers.Dense(4, activation="relu"))
modelr.add(layers.Dense(1,  activation="sigmoid"))

modelr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
modelr.fit(X_train, Y_train, shuffle=False, batch_size=1, epochs=1, validation_data=(X_test, Y_test))

loss, accuracy = modelr.evaluate(X_test, Y_test, verbose=0)
    
print(loss, accuracy*100)

3348/3348 [==============================] - 5s 1ms/step - loss: 0.1066 - accuracy: 0.9731 - val_loss: 0.0687 - val_accuracy: 0.9892


(1.3302180767059326, 0.7325581312179565)